In [5]:
import openai
import gsearch
import json
import os
import tiktoken
import time
import inspect
import callgpt
import react_chain
import compliance_checker
import excel_analysis
import agents_report
import agents_discussion
import repo_reader
import read_file
import re
import tkinter as tk
from tkinter import ttk
from openpyxl import load_workbook
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter import filedialog
from tkinter import *

def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(text))
    return num_tokens
    
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
os.environ["OPENAI_API_KEY"] = open_file('Keys/openai_api_key.txt')
openai.api_key = open_file('Keys/openai_api_key.txt')
openai_api_key = openai.api_key
callgpt = callgpt.Ask()
current_dir = os.getcwd()
os.chdir(current_dir)

class BusinessAnalyst:
    def __init__(self):
        self.role_system = {"role": "system", "content": "You are a highly organised and detail oriented Goldman Sachs business analyst."}
        self.messages = [self.role_system]
        # INSERT INFO ABOUT SAVING THE PROMPT + COMPLETION PAIRS - FAISS? CHROMA?

    def write_output_to_file(self, output, file_name='analyst_output.txt', mode='a'):
        with open(file_name, mode, encoding='utf-8') as f:
            f.write(f"{output} \n")

    def chat_with_gpt3(self, prompt, max_retries=3, delay=2):
        user_message = {"role": "user", "content": prompt}
        new_message_tokens = count_tokens(prompt)
        total_tokens = sum(count_tokens(m["content"])
                        for m in self.messages) + new_message_tokens

        while total_tokens + 1 > 3500:
            # Remove the oldest user-assistant message pair to stay within the token limit
            self.messages.pop(1)
            self.messages.pop(1)
            total_tokens = sum(count_tokens(m["content"])
                            for m in self.messages) + new_message_tokens

        self.messages.append(user_message)
        response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages
                )

        # Retry logic
        retries = 0
        while retries < max_retries:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages,
                )
                assistant_message = {
                    "role": "assistant", "content": response.choices[0].message["content"]}
                self.messages.append(assistant_message)
                return assistant_message["content"]
            except openai.error.OpenAIError as e:
                print(f"Error occurred during API call: {e}")
                retries += 1
                if retries < max_retries:
                    time.sleep(delay)  # Delay between retries
                else:
                    raise


    def process_input(self, user_query):
        prompt = f"You are an AI who wants to answer the following user query: '{user_query}' with a numbered list of tasks. Response: "
        processed_query = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Processed query:*****\n" + "\033[0m\033[0m"+processed_query)
        self.write_output_to_file(processed_query)  # Write the output to the file
        return processed_query

    # Not strictly speaking necessary, unless we want to do nested tasks
    def decompose_tasks(self, processed_query): 
        prompt = f"You are an AI whose job is to assess the following processed query into the most feasible first task to start with and create a new numbered task list: '{processed_query}'. Response: "
        sub_tasks = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Subtasks*****\n"+"\033[0m\033[0m" + sub_tasks)
        self.write_output_to_file(sub_tasks)
        return sub_tasks

    def select_task(self, sub_tasks):
        prompt = f"Select the first task to perform from '{sub_tasks}'. Response: "
        selected_task = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Selected Tasks:*****\n"+"\033[0m\033[0m"+ (selected_task))
        self.write_output_to_file(selected_task)
        return selected_task

    def choose_best_tool(self, task):
        prompt = f"""You are an AI that has to select the best tool to use to solve the following task: '{task}'.
        Please respond in the format 'Action: [tool_name] only including items from this list - Action: (gsearch|repo_reader|ask_user_input|agent_discussion|agent_report|analyse_excel.
        This is how you make your choice:
        If you think talking it through might help: agent_discussion
        If you want to write a report: agent_report
        If you need to seach the web: gsearch
        If you need user input: ask_user_input
        The response should contain absolutely no other words. 
        The default option should be 'Action: gsearch' """
        response = self.chat_with_gpt3(prompt)
        tool_name = response.split(': ')[1]  # Split the response and retrieve the tool name

        print(f"GPT-3 Response: {response}")
        self.write_output_to_file(f"GPT-3 Response: {response}")  
        tool_re = re.compile(r"'\s*(.*?)\s*'")
        tool_match = tool_re.search(response)
        if not tool_match:
            tool_re = re.compile(r"Action:\s*(gsearch|repo_reader|ask_user_input|agent_discussion|agent_report|analyse_excel)", re.IGNORECASE)
            tool_match = tool_re.search(response)

        if tool_match:
            best_tool = tool_match.group(1)
            print(f"Extracted tool from GPT-3 response: {best_tool}")
            self.write_output_to_file(f"Extracted tool from GPT-3 response: {best_tool}")
            return best_tool.strip().lower().replace(" ", "_")
        else:
            print(f"Unexpected GPT-3 response: {response}")
            self.write_output_to_file(f"Unexpected GPT-3 response: {response}")
            return self.user_select_tool()

    def user_select_tool(self):
        print("Unexpected GPT-3 response. Please select the next task to perform:")
        tools = ["gsearch", "repo_reader", "ask_user_input", "agent_discussion", "agent_report", "analyse_excel"]

        for i, tool in enumerate(tools, start=1):
            print(f"{i}. {tool}")
            self.write_output_to_file(f"Unexpected GPT-3 response. Please select the next task to perform: {i}. {tool}")

        while True:
            try:
                user_choice = int(input("Enter the number of your choice: "))
                if 1 <= user_choice <= len(tools):
                    return tools[user_choice - 1]
                else:
                    print("Invalid choice. Please enter a number between 1 and 6.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def ask_user_input(self, prompt):
        user_input = input("What should I do\n" + prompt)
        self.write_output_to_file(f"What should I do re {prompt} and {user_input}")
        result = self.chat_with_gpt3(f"re {prompt} I think we should do {user_input}")
        return result

    def agent_discussion(self, prompt):
        discussion = agents_discussion.generate_output(prompt)
        self.write_output_to_file(discussion)
        return discussion

    def agent_report(self, prompt):
        report = agents_report.create_joint_letter(prompt)
        self.write_output_to_file(report)
        return report
    
    def execute_tasks(self, task, max_retries=3):
        results = []
        # ADD MORE FUNCTIONS
        function_mapping = {
            "gsearch": gsearch.execute,
            "repo_reader": repo_reader.main,
            "analyse_excel": self.analyse_excel,
            "ask_user_input": self.ask_user_input,
            "agent_discussion": self.agent_discussion,
            "agent_report": self.agent_report
            # "scrape_site": main.scrape
        }

        retries = 0
        while retries < max_retries:
            best_tool = self.choose_best_tool(task)
            if best_tool in function_mapping:
                tool_function = function_mapping[best_tool]
                print(f"Tool Function is {best_tool}")
                break
            else:
                print(f"Invalid tool selected: {best_tool}. Retrying...")
                retries += 1

        if retries == max_retries:
            raise ValueError("Failed to select a valid tool after multiple retries.")

        prompt = f"{best_tool}: {task}"
        result = tool_function(task)  # Use the selected function to solve the task

        def validate_output(output):
            nonlocal user_validation
            user_validation = True
            validation_window.destroy()

        def reject_output():
            nonlocal user_validation
            user_validation = False
            validation_window.destroy()

        user_validation = False
        while not user_validation:
            validation_window = tk.Tk()
            validation_window.title("Task Output Validation")
            tk.Label(validation_window, text="Task Output:").grid(row=0, column=0)
            tk.Label(validation_window, text=result).grid(row=1, column=0)
            tk.Button(validation_window, text="Accept Output", command=validate_output).grid(row=2, column=0)
            tk.Button(validation_window, text="Reject Output", command=reject_output).grid(row=2, column=1)
            validation_window.mainloop()

            if not user_validation:
                result = tool_function(prompt)

        print(f"\033[92m\033[1m" + "\n*****Results Of Execution*****\n" + "\033[0m\033[0m {result}")
        self.write_output_to_file(f"\n*****Results Of Execution*****\n {result}")
        return result

    def analyze_results(self, results):
        prompt = f"You are an AI focused on analysing the results from executing the task: '{results}'. Please critically explain and appraise the output. Answer what are avenues to further explore and understand from this. "
        analysis = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Analysis*****\n"+"\033[0m\033[0m" + (analysis))
        self.write_output_to_file("\n*****Analysis*****\n" + analysis)
        return analysis

    def refine_tasks(self, analysis):
        prompt = f"Considering the results, let's refine and create a new list of tasks based on the following analysis: {analysis}. The response should be in a numbered list with one sentence each. Response: "
        new_tasks = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Refined New Tasks*****\n"+"\033[0m\033[0m"+ (new_tasks))
        self.write_output_to_file("\n*****Refined New Tasks*****\n" + new_tasks)
        return new_tasks

    def generate_output(self, analysis):
        prompt = f"You are an analyst who will create a readable document from the whole discussion summarising it and clearly articulating key theories, ideas and actions to be done. Please generate a user-friendly output based on the following analysis: {analysis}. Response: "
        output = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Generated Output*****\n"+"\033[0m\033[0m"+ (output))
        self.write_output_to_file("\n*****Generated Output*****\n" + output)
        return output

    def run_analysis(self, user_query):
        processed_query = self.process_input(user_query)
        sub_tasks = self.decompose_tasks(processed_query)
        selected_tasks = self.select_task(sub_tasks)
        results = self.execute_tasks(selected_tasks)
        analysis = self.analyze_results(results)
        new_tasks = self.refine_tasks(analysis)
        ques = input("Do you want to continue - Y or N? ")
        while ques == "Y":
            sub_tasks = self.process_input(new_tasks)
            selected_tasks = self.select_task(sub_tasks)
            results = self.execute_tasks(selected_tasks)
            analysis = self.analyze_results(results)
            new_tasks = self.refine_tasks(analysis)
            ques = input("Do you want to continue - Y or N? ")
        if ques == "N":
            print("\033[92m\033[1m" + "\n*****Alright! Final Results Coming Up*****\n" + "\033[0m\033[0m")
            self.write_output_to_file("\n*****Alright! Final Results Coming Up*****\n")
        else:
            print("\n\n*******Dude! I said Y or N!*******\n\n")
        output = self.generate_output(analysis)
        return output
    
    def get_excel_file_path(self):
        root = Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(initialdir="/", title="Select a File", filetypes=[("Excel Files", "*.xlsx")])
        print(file_path)
        root.destroy()
        return file_path

    def analyse_excel(self, prompt):
        print("\033[92m\033[1m" + "\n*****Excel Analysis*****\n" + "\033[0m\033[0m")
        self.write_output_to_file("\n*****Excel Analysis*****\n")
        functions_list = [func for func in inspect.getmembers(excel_analysis, inspect.isfunction)] # Use if needed
        filepath = self.get_excel_file_path()
        results = excel_analysis.main(filepath)
        min_acceptable_rating = 7
        for recursion_count in range(3):
            analysis = self.analyze_results(results)
            prompt = f"As a top analyst at Goldman Sachs, when presented with an Excel sheet with a companys financial statements, define what tests and hypotheses would you employ to determine if the company is investable? {analysis}. Write your response only as a numbered task list: "
            output = self.chat_with_gpt3(prompt)

            # Evaluate the output
            evaluation_prompt = f"How good is the following analysis as a numbered list of ways to analyse an excel sheet: '{output}'. Please rate it on a scale of 1 to 10, with 10 being the best: "
            rating = self.chat_with_gpt3(evaluation_prompt)
            print(f"\n**Rating Is** {rating}")
            # Extract the first number found in the response
            number_match = re.search(r'\d+', rating)
            if number_match:
                rating = int(number_match.group())
            else:
                rating = 1  # Set the rating to 1
            if rating >= min_acceptable_rating:
                break
            else:
                prompt = f"{prompt} Please provide a more detailed and accurate analysis."

        sub_tasks = self.process_input(f"Given this, how would you go about answering Question:{prompt}. This is the Output re the excel structure:{output}")

        final_results = []
        for task in sub_tasks:
            print("\n**My First Task Is: {task}")
            result = self.execute_tasks(task)
            final_results.append(result)

        # Combine the results and analyze them
        final_result = ' '.join(final_results)
        analysis = self.analyze_results(final_result)

        return final_result


ba = BusinessAnalyst()
result = None

while True:
    try:
        user_query = input("""Choose one. What would you like to do? \n
                    1. Ask something you're curious about, run tasks \n
                    2. Figure out something about your docs \n
                    3. Figure out what's inside a python repo\n
                    4. Simulate a dialogue between two people \n
                    5. Let's write a report together! \n
                    6. Analyse an excel! \n
                    7. Read a document and return text. \n
                    00. Exit the prompt \n
                    """)
        if user_query == "1":
            prompt = input("Enter your query: ")
            print(f"""\033[92m\033[1m" + "\n*****Your Question*****\n" + "\033[0m\033[0m {prompt}""")
            ba.write_output_to_file(f"\n*****Your Question*****\n is {prompt}")
            result = ba.run_analysis(prompt)
        elif user_query == "2":
            # Call the function to figure out something about your docs
            # Replace this line with the appropriate function call
            result = "\n\nFigure out something about your docs (function not implemented)\n\n"
        elif user_query == "3":
            # Call the function to figure out what's inside a python repo
            # Replace this line with the appropriate function call
            result = "\n\nFigure out what's inside a python repo (function not implemented)\n\n"
        elif user_query == "4":
            # Call the function to simulate a dialogue between two people
            import agents_discussion
            result = agents_discussion.generate_output(user_query)
            print("\n\nHere's a dialogue!\n\n")
        elif user_query == "5":
            # Call the function to write a report together
            import agents_report
            result = agents_report.create_joint_letter(user_query)
            print("\n\nHere's the report!\n\n")
        elif user_query == "6":
            # Call the function to analyze an excel
            result = ba.analyse_excel("Analyze an excel sheet")
        elif user_query == "7":
            fp = filedialog.askopenfilename(initialdir="/", title="Select a File")
            result = read_file.read_file(fp)
        elif user_query == "00":
            result = "\n\nSorry to see you go !!\n\n"
        else:
            print("\n\nInvalid option. Please choose a number between 1 and 7.\n\n")
    except TypeError as e:
        print(f"An error occurred: {e}\nPlease try again.")
    except ValueError as e:
        print(f"An error occurred: {e}\nPlease try again.")

    if result:
        print(result)
        ba.write_output_to_file(result)


*****Excel Analysis*****



2023-04-17 16:49:40.402 Python[70246:6197612] +[CATransaction synchronize] called within transaction


/Users/rohit/Downloads/Apple-Income-Statement.xlsx

Sheet hierarchy and structure:
 - Income Statement

Named ranges:

Sheet contents:

Sheet: Income Statement
Column names:
 - A: Apple Inc.

Formulae:

Sheet: Income Statement
Formulas:

Data flows and dependencies:

Sheet: Income Statement

External links:

Key outputs and their dependencies:


*****Analysis*****
As an AI, I can analyze the given task output and provide a critical explanation and appraisal of the same.

Firstly, the given output focuses on the "Income Statement" sheet hierarchy and structure with named ranges, sheet contents, column names, and formulas. However, the output is incomplete as there is no information about the named ranges, sheet contents, and formulas of the sheet mentioned. Without this information, it is tough to determine the accuracy and completeness of the information provided in the sheet.

Secondly, the data flows and dependencies are not given in the output, which makes it hard to understand the 

KeyboardInterrupt: Interrupted by user